# **Primer análisis de datos extraidos**

#### Incluimos las librerías necesarias para analizar los datos

In [12]:
import pandas as pd
import json
from collections import defaultdict
import re
from datetime import datetime as dt, timedelta as td

#### Leemos los datos del JSON y lo guardamos en la variable *datos*

In [2]:
datos = pd.read_json('traces_Articoding_Escolapias.json')

#### Usamos el metodo **head** para ver como estan estructurados los elementos del JSON

In [3]:
datos.head(10)

,actor,result,verb,_id,object,timestamp
0,"{'name': 'gqoj', 'account': {'homePage': 'http...",{'extensions': {'content': ': '}},{'id': 'http://adlnet.gov/expapi/verbs/initial...,628179b5a6ba41003d3bada2,{'definition': {'type': 'https://w3id.org/xapi...,2022-05-16 10:36:35.361000+00:00
1,"{'name': 'gqoj', 'account': {'homePage': 'http...",{'extensions': {'https://w3id.org/xapi/serious...,{'id': 'http://adlnet.gov/expapi/verbs/progres...,628179b5a6ba41003d3bada2,{'definition': {'type': 'https://w3id.org/xapi...,2022-05-16 10:36:35.317000+00:00
2,"{'name': 'gqoj', 'account': {'homePage': 'http...","{'extensions': {'fullscreen': True, 'language'...",{'id': 'http://adlnet.gov/expapi/verbs/initial...,628179b5a6ba41003d3bada2,{'definition': {'type': 'https://w3id.org/xapi...,2022-05-16 10:36:35.316000+00:00
3,"{'name': 'gqoj', 'account': {'homePage': 'http...",NaN,{'id': 'https://w3id.org/xapi/seriousgames/ver...,628179b5a6ba41003d3bada2,{'definition': {'type': 'https://w3id.org/xapi...,2022-05-16 10:36:35.297000+00:00
4,"{'name': 'gqoj', 'account': {'homePage': 'http...",{'extensions': {'scene': 'menu'}},{'id': 'https://w3id.org/xapi/seriousgames/ver...,628179b5a6ba41003d3bada2,{'definition': {'type': 'https://w3id.org/xapi...,2022-05-16 10:36:42.439000+00:00
5,"{'name': 'gqoj', 'account': {'homePage': 'http...",{'extensions': {'content': 'Estrellas por cate...,{'id': 'http://adlnet.gov/expapi/verbs/initial...,628179b5a6ba41003d3bada2,{'definition': {'type': 'https://w3id.org/xapi...,2022-05-16 10:36:42.435000+00:00
6,"{'name': 'gqoj', 'account': {'homePage': 'http...","{'score': {'raw': 1}, 'extensions': {'content'...",{'id': 'http://adlnet.gov/expapi/verbs/complet...,628179b5a6ba41003d3bada2,{'definition': {'type': 'https://w3id.org/xapi...,2022-05-16 10:36:42.434000+00:00
7,"{'name': 'gqoj', 'account': {'homePage': 'http...",{'extensions': {'content': 'Menús: Puedes eleg...,{'id': 'http://adlnet.gov/expapi/verbs/initial...,628179b5a6ba41003d3bada2,{'definition': {'type': 'https://w3id.org/xapi...,2022-05-16 10:36:40.688000+00:00
8,"{'name': 'gqoj', 'account': {'homePage': 'http...","{'score': {'raw': 1}, 'extensions': {'content'...",{'id': 'http://adlnet.gov/expapi/verbs/complet...,628179b5a6ba41003d3bada2,{'definition': {'type': 'https://w3id.org/xapi...,2022-05-16 10:36:40.685000+00:00
9,"{'name': 'gqoj', 'account': {'homePage': 'http...",{'extensions': {'content': ': '}},{'id': 'http://adlnet.gov/expapi/verbs/initial...,628179b5a6ba41003d3bada2,{'definition': {'type': 'https://w3id.org/xapi...,2022-05-16 10:36:46.017000+00:00


## Obtenemos el codigo de inicio y find e nivel

In [4]:
prueba = datos.head(20)

tiempos = set()   
#Buscamos el nombre de los tiempos de inicio y fin, los cuales se encuentran en 'verb'
for time in prueba['verb']:
    tiempos.add(time['id'])
#Guardamos el link de inicio y fin
print(tiempos)
initialized = "http://adlnet.gov/expapi/verbs/initialized"
completed = "http://adlnet.gov/expapi/verbs/completed"
print('Initialized: '+ initialized,'Completed: '+ completed)

{'http://adlnet.gov/expapi/verbs/progressed', 'http://adlnet.gov/expapi/verbs/initialized', 'https://w3id.org/xapi/seriousgames/verbs/accessed', 'http://adlnet.gov/expapi/verbs/completed'}
Initialized: http://adlnet.gov/expapi/verbs/initialized Completed: http://adlnet.gov/expapi/verbs/completed


## Creamos una lista de diccionarios con los timepos de cada usuario en cada nivel

In [15]:
#verb nos dice si se ha iniciado o completado un nivel
#Buscamos guardar en un diccionario de usuarios diccionarios de niveles con los tiempos de ejecución
userList = defaultdict(defaultdict)
tam = len(datos)
for elemento in datos.to_numpy():
    #Comprobamos que la acción sea de inicio o fin de nivel
    verb = elemento[2]['id']
    #Diccionario de usuarios --> 
    if verb == initialized or verb == completed:
        #Extraemos el nombre del usuario y el nivel
        name = elemento[0]['name']
        timestamp = elemento[5]
        level = elemento[4]['id']
        #Calculamos el tiempo que tarda en terminar el nivel
        if level in userList[name]:
            userList[name][level][verb] = timestamp
        else:
            userList[name][level] = {verb: timestamp}

            
        

userList

defaultdict(collections.defaultdict,
            {'gqoj': defaultdict(None,
                         {'https://simva-api.simva.e-ucm.es/activities/628179b5a6ba41003d3bada2/tip_0_bienvenido': {'http://adlnet.gov/expapi/verbs/initialized': Timestamp('2022-05-19 12:13:42.333000+0000', tz='UTC'),
                           'http://adlnet.gov/expapi/verbs/completed': Timestamp('2022-05-19 12:13:48.985000+0000', tz='UTC')},
                          'https://simva-api.simva.e-ucm.es/activities/628179b5a6ba41003d3bada2/articoding': {'http://adlnet.gov/expapi/verbs/initialized': Timestamp('2022-05-23 15:51:08.085000+0000', tz='UTC')},
                          'https://simva-api.simva.e-ucm.es/activities/628179b5a6ba41003d3bada2/tip_4_estrellas': {'http://adlnet.gov/expapi/verbs/initialized': Timestamp('2022-05-19 12:13:56.635000+0000', tz='UTC'),
                           'http://adlnet.gov/expapi/verbs/completed': Timestamp('2022-05-19 12:14:00.267000+0000', tz='UTC')},
                    

## Tiempo total de juego de cada jugador

In [29]:

from datetime import timedelta


for user in userList:
    time = timedelta(0)
    for level in userList[user]:
        if(len(userList[user][level]) == 2):
            time += (userList[user][level][completed]-userList[user][level][initialized])
    print("Usuario: "+ user)
    print("Tiempo de juego: ", time)


Usuario: gqoj
Tiempo de juego:  4 days 03:59:27.108000
Usuario: hgfx
Tiempo de juego:  0 days 00:02:16.608000
Usuario: gton
Tiempo de juego:  0 days 00:00:09.296000
Usuario: uala
Tiempo de juego:  0 days 00:02:26.634000
Usuario: qeqyj
Tiempo de juego:  0 days 01:57:38.175000
Usuario: fkmtp
Tiempo de juego:  0 days 03:30:19.480000
Usuario: hjpbw
Tiempo de juego:  0 days 02:54:33.745000
Usuario: bdwyj
Tiempo de juego:  0 days 03:03:40.201000
Usuario: eprdu
Tiempo de juego:  0 days 03:33:44.800000
Usuario: ftpnb
Tiempo de juego:  0 days 03:30:19.575000
Usuario: ikcar
Tiempo de juego:  0 days 02:53:47.522000
Usuario: pbgky
Tiempo de juego:  0 days 02:45:10.748000
Usuario: gleqe
Tiempo de juego:  0 days 03:34:55.667000
Usuario: gcpze
Tiempo de juego:  0 days 03:36:59.763000
Usuario: loemd
Tiempo de juego:  0 days 02:00:01.011000
Usuario: ejeuw
Tiempo de juego:  0 days 02:59:44.546000
Usuario: srjon
Tiempo de juego:  0 days 02:02:56.651000
Usuario: sxipu
Tiempo de juego:  0 days 02:47:29.904

## Tiempo medio para completar cada nivel